# Project 3


## Instructions

In this project, you will solve a two-dimensional, two-phase flow (i.e. oil and water) reservoir simulation in a heterogenuous reservoir with multiple wells.  Essentially, all of the functionality needed to do this was already implemented in [Homework Assignment 20](https://github.com/PGE323M-Fall2018/assignment20).  We will use real data from the Nechelik reservoir that we have looked at several times throughout the semester.

For this project, you should implement the class below `Project3()` which inherits from `TwoPhaseFlow` (which inherits from `TwoDimReservoir` and `BuckleyLevertt`).  

You will need to implement some functionality to read the porosity and permeability information from a file as you did in [Project2](https://github.com/PGE323M-Fall2018/project2).  You will notice in [inputs.yml](inputs.yml), that these values take the filenames [`Nechelik_perm.dat`](Nechelik_perm.dat) and [`Neckelik_poro.dat`](Nechelik_poro.dat).  These files have the permeability and porosity data, respectively, for each grid block. (You probably have already updated your [Homework Assignment 17](https://github.com/PGE323M-Fall2018/assignment17) files to include this functionality and not need to do this again.)
 
Other than reading the data from a file, you may not need to write any additional code for your simulation to work.  However, it might be a good idea to write a few plotting routines to produce some plots like this one

![img](images/contour.png)

to help you determine if your code is working correctly.

As you know, the actual Nechelik field has an irregular geometry as shown in the figure, with maximum $d = 100$ ft, $h = 5753$ ft and maximum $L = 7060.5$ ft. There are $N = 1188$ values in the data files corresponding to $N_x$ = 54 and $N_y$ = 22 grids to be used in the reservoir.  The reservoir has constant properties $\mu_w = \mu_o = 1$ cp, $B_w = B_o = 1$, $c_o = c_w = 1 \times 10^{-5}$ psi$^{-1}$, an inital reservoir pressure of $p_{\mbox{initial}} = 3700$ psi, and an initial water satruation $S_{wi} = 0.2$.  The Corey-Brooks properties are included in the [inputs.yml](inputs.yml) file.

The reservoir has the following wells

|**Well**|**Location**<br> (ft, ft)|**Well type** | **Operating conditions** <br> (ft$^3$/day or psi)|
|:-:|:-:|:-:|:-:|
|1| 5536, 3500| Constant BHP | 2000 |
|2| 5474, 4708| Constant BHP | 2000 |
|3| 3600, 4937| Constant BHP | 2000 |
|4| 2400, 3322| Constant BHP | 2000 |
|5| 2500, 4050| Constant rate water injector | 1000 |

All wells have a radius of $r_w = 0.25$ ft and negligible skin factor.

### Suggestion

 * Because the file [`Neckelik_poro.dat`](Nechelik_poro.dat) has zero porosity values for those grids outside the true reservoir domain, you may get a "divide by 0" error when updating the saturations with the explicit equation.  Please take a look at this [StackOverflow answer](https://stackoverflow.com/questions/26248654/numpy-return-0-with-divide-by-zero) for a hint at how to avoid this.

## Testing

There are no locally available tests for this project, but if your `TwoPhaseFlow` class passed all tests from [Homework Assignment 20](https://github.com/PGE323M-Fall2018/assignment20) you can be reasonably assured it will work correctly.  Tests will be run on Travis and you will recieve feedback on whether they are passing or not upon submission. You can continue to resubmit until the deadline.

I encourage you to come up with your own tests as well.  One thing you might do is change the Corey-Brooks parameters to mimic single-phase behavior and set the initial water satruation to $S_{wi} = 1.0$ and compare your results to the results of [Project2](https://github.com/PGE323M-Fall2018/project2).  While I have not worked a complete tutorial for this project in CMG, I did record a tutorial for the solving the Buckley-Leverett problem in CMG here: http://youtu.be/zuCHYYxsFQg.  If you combine what you learn in this tutorial with your work from [Homework Assignment 18](https://github.com/PGE323M-Fall2018/assignment18), you should be able to solve this project in CMG.  **Please Note:** Unlike the single-phase examples we've compared previously, there may be small differences in what your project code produces and the results of CMG.  This is due to the fact that we have implemented an IMPES formulation whereas CMG uses a fully implicit solution scheme.  However, they should be very close, especially for early times.

## Backup Plan

If you cannot get this project to work, you may work the project in CMG for 1/2 credit.  If you choose to do this, please add your CMG input file, names as `project3.dat` and export the pressures at the end of 200 days to a file `pressure.txt` and the saturations to a file `saturation.txt`.  Add the three files to this repository and upload it to Github.  These will be graded manually.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

In [2]:
from assignment20 import TwoPhaseFlow

class Project3(TwoPhaseFlow):      
            
    def __init__(self, inputs):
        
        super().__init__(inputs)
        
    def check_input_and_return_data(self, input_name):
        dtype = type(input_name)
            
        if dtype == str:
            filename = input_name
            data = np.loadtxt(filename)
        elif dtype == list or dtype == tuple:
            data = np.array(input_name)  
            
        elif dtype == int or dtype == float:
                ngrids = self.ngrids
                data = (input_name * np.ones(ngrids))
        else:
            raise ValueError('Error - wrong data type: %s' % (input_name))
            
        return data